In [1]:
from httpx import AsyncClient
import asyncio
import bs4
import asyncio
import httpx
import bs4
import pandas as pd
import aiohttp
import geopandas
##

In [2]:
#gen empresas
df_bruto = pd.read_csv("/home/gabri/plano_diretor_poc/empresas_natjur_cnae.csv")

#create new column concatenating tipo_logradouro, logradouro, numero, bairro, 'BELÉM - PA'
#! Modificar se necessário
df_bruto['endereco_url'] = 'https://www.google.com/maps/search/'+ df_bruto['razao_social'] +'+' + df_bruto['tipo_logradouro'] +'+' +  df_bruto['logradouro'] +'+' +  df_bruto['numero'] +'+' +  df_bruto['bairro'] +'+' +  'BELÉM+PA'

#remove nan endereco_url from df
df = df_bruto.dropna(subset=['endereco_url'])
cnpj_endereco_url = dict((zip(df['cnpj'], df['endereco_url'])))

In [3]:
#podemos fazer a instalação da aiohttp para fazer requisições HTTP assíncronas com pip install aiohttp.
#crawler 
async def fetch_page(session, cnpj, url):
    async with session.get(url) as response:
        if response.status != 200:
            return cnpj, None  # Retorna None se a resposta não for bem-sucedida
        html = await response.text()
        soup = bs4.BeautifulSoup(html, 'html.parser')
        try:
            lat_long = soup.find('meta', {'itemprop': 'image'}).get('content').split('center=')[1].split('&')[0].split('%2C')
        except Exception as e:
            lat_long = None
        return cnpj, lat_long


async def fetch_all_pages():
    async with aiohttp.ClientSession() as session:
        tasks = []
        for cnpj, url in cnpj_endereco_url.items():
            tasks.append(fetch_page(session, cnpj, url))
        results = await asyncio.gather(*tasks, return_exceptions=True)  # Lidando com exceções
        # Construir um dicionário com os resultados
        result_dict = {cnpj: info for cnpj, info in results if info is not None}
        return result_dict
    
async def main():
    pages = await fetch_all_pages()
    
    return pages


results = await main()

In [4]:
#-- converter para df
df = pd.DataFrame.from_dict(results, orient='index', columns=['lat', 'long']).reset_index()
df.columns = ['cnpj', 'lat', 'long']

In [5]:
#-- left join df_bruto on df
df = df.merge(df_bruto, on='cnpj', how='left')

#-- converter para pd.DataFrame com uma coluna WKT (well known text) para plotar no QGIS
gdf = geopandas.GeoDataFrame(
    df, geometry=geopandas.points_from_xy(df.long, df.lat), crs="EPSG:4326"
)
#
#save gdf to .shp
gdf.to_file("/home/gabri/plano_diretor_poc/edificios_belem_08_02_2024.shp")
#save to csv
gdf.to_csv("/home/gabri/plano_diretor_poc/edificios_belem_08_02_2024.csv")


/tmp/ipykernel_2673/2008209351.py:10: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file("/home/gabri/plano_diretor_poc/edificios_belem_08_02_2024.shp")
